In [1]:
import pandas as pd
from ast import literal_eval

#### load raw parsed json html file

In [2]:
df=pd.read_csv('./new_golden.csv')
df.shape

(119819, 38)

#### remove null

In [6]:
df = df[~df['username'].isnull()]
df.shape

(108016, 38)

#### remove duplicates

In [15]:
df = df[~df['username'].duplicated()]

In [56]:
pd.options.display.max_columns=100
df.head(3)

,about.avatar,about.avatar_cdn,about.avatar_large,about.avatar_large_cdn,about.chosen_role,about.company,about.corporate,about.cover_image,about.display_name,about.hourly_rate,about.id,about.location,about.preferred_freelancer,about.primary_currency,about.primary_language,about.profile_description,about.public_name,about.recommendations,about.registration_date,about.registration_date_human,about.reputation,about.role,about.status,about.tagline,about.timezone,about.username,about.username_sentence,hourlyRate,resume.certifications,resume.education,resume.exams,resume.experience,resume.publications,resume.show,skills,timeOffset,userId,username
0,/ppic/1869150/logo/1018449/profile_logo_101844...,//cdn2.f-cdn.com/ppic/1869150/logo/1018449/pro...,/ppic/1869149/logo/1018449/profile-image-250.png,//cdn3.f-cdn.com/ppic/1869149/logo/1018449/pro...,2.0,itsol,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,techit99,60,1018449.0,"{u'administrative_area': None, u'city': u'Bris...",False,"{u'code': u'USD', u'name': u'US Dollar', u'cou...",en,NaN,techit99,0.0,1.237934e+09,"March, 2009","{u'user_id': 1018449, u'last3months': {u'compl...",1.0,"{u'payment_verified': False, u'identity_verifi...",NaN,"{u'country': u'UK', u'offset': 1, u'id': 262, ...",techit99,Techit99,60.0,"[{u'award_date': u'2011', u'descr': u'Hootsuit...",[],[],"[{u'year_interval': u'-2011', u'month_interval...",[],True,"[{u'usages': 0, u'seo_url': u'php', u'islocal'...",0.0,1018449.0,techit99
1,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,1.0,Handmade Cards for Animal Charities,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,jung14,25,1344962.0,"{u'administrative_area': None, u'city': u'Penr...",False,"{u'code': u'USD', u'name': u'US Dollar', u'cou...",en,NaN,jung14,0.0,1.262802e+09,"January, 2010","{u'user_id': 1344962, u'last3months': {u'compl...",0.0,"{u'payment_verified': False, u'identity_verifi...",NaN,"{u'country': u'UK', u'offset': 1, u'id': 262, ...",jung14,Jung14,25.0,[],[],[],[],[],False,"[{u'usages': 0, u'seo_url': u'copywriting', u'...",0.0,1344962.0,jung14
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df.columns[-40:]

Index([u'about.avatar', u'about.avatar_cdn', u'about.avatar_large',
       u'about.avatar_large_cdn', u'about.chosen_role', u'about.company',
       u'about.corporate', u'about.cover_image', u'about.display_name',
       u'about.hourly_rate', u'about.id', u'about.location',
       u'about.preferred_freelancer', u'about.primary_currency',
       u'about.primary_language', u'about.profile_description',
       u'about.public_name', u'about.recommendations',
       u'about.registration_date', u'about.registration_date_human',
       u'about.reputation', u'about.role', u'about.status', u'about.tagline',
       u'about.timezone', u'about.username', u'about.username_sentence',
       u'hourlyRate', u'resume.certifications', u'resume.education',
       u'resume.exams', u'resume.experience', u'resume.publications',
       u'resume.show', u'skills', u'timeOffset', u'userId', u'username'],
      dtype='object')

#### memove fields with no value, e.g., profile pic link, profile duplicate ID, fields has 95% more all null values

In [17]:
df1 = df.drop(['about.avatar', u'about.avatar_cdn', u'about.avatar_large',
       u'about.avatar_large_cdn', u'about.cover_image', u'about.id',
                'about.corporate', 'about.preferred_freelancer', 
                ], axis=1)

* extract month and year as new fields from registration date 
* convert profile_description field to length of characters

In [18]:
df1['register_Month'] = df1['about.registration_date_human'].fillna('NoMn, NoYr').apply(lambda x: x.split(',')[0] )
df1['register_Year'] = df1['about.registration_date_human'].fillna('NoMn, NoYr').apply(lambda x: x.split(',')[1] )
df1['desc_len'] = df1['about.profile_description'].apply(lambda x: len(str(x).strip()))
# df1['about.location'] = df1['about.location'].fillna("")

In [19]:
df2 = df1.drop(['about.registration_date_human','about.profile_description'], axis=1)
df2.shape

(105880, 31)

* helper function that unpack dictionary

In [22]:
def flatten_dict(data, layers=2, drop_deeper=True):

        for _ in range(layers):
            data = [(k, v) if not isinstance(v, dict) else [(k + '_' + k2, v2) for k2, v2 in v.items()] for k, v in
                    data.items()]
            data = [item for sublist in data for item in sublist if isinstance(sublist, list)] + [y for y in data if
                                                                                                  not isinstance(y,
                                                                                                                 list)]
            data = dict(data)

        if drop_deeper:
            data = {k: v for k, v in data.items() if not isinstance(v, dict) or isinstance(v, list)}

        return data
    
def mapping(x):
    if type(x)==str:
        
        try:
            tmp = flatten_dict(literal_eval(x))
        except:
            tmp=None
    else:
        tmp=x
    return tmp

def retrieve(field, df=df):
    one = df[field].apply(mapping)
    two = pd.DataFrame.from_dict(one.to_dict())
    three = two.transpose()
    return three

In [23]:
df2.shape

(105880, 31)

## unpack dictionary fields

### location

In [24]:
%%time
location = retrieve('about.location')
print location.shape

(105880, 22)
CPU times: user 22.6 s, sys: 340 ms, total: 22.9 s
Wall time: 22.9 s


#### the step below is to add a '_n' to the end of each field in case the unpacked field has the same name of the main table

In [25]:
location.columns=map(lambda x: str(x)+'_3',location.columns)

#### ps, since fields like country_flag_url, each country has the same url, no need to remove the field
#### ps, but I'll drop those fields later due to duplicate infomation

In [46]:
location['country_flag_url_3'].value_counts().head()

/img/flags/png/us.png    29508
/img/flags/png/in.png    20694
/img/flags/png/gb.png     7360
/img/flags/png/pk.png     4278
/img/flags/png/ca.png     3901
Name: country_flag_url_3, dtype: int64

In [26]:
location.head()

,administrative_area_3,city_3,country_code_3,country_demonym_3,country_flag_class_3,country_flag_url_3,country_flag_url_cdn_3,country_highres_flag_url_3,country_highres_flag_url_cdn_3,country_iso3_3,...,country_name_3,country_person_3,country_phone_code_3,country_region_id_3,country_sanction_3,country_seo_url_3,full_address_3,latitude_3,longitude_3,vicinity_3
0,None,Bristol,gb,None,united-kingdom,/img/flags/png/gb.png,//cdn6.f-cdn.com/img/flags/png/gb.png,/img/flags/highres_png/united-kingdom.png,//cdn5.f-cdn.com/img/flags/highres_png/united-...,None,...,United Kingdom,None,None,None,None,None,None,None,None,None
1,None,Penrith,gb,None,united-kingdom,/img/flags/png/gb.png,//cdn6.f-cdn.com/img/flags/png/gb.png,/img/flags/highres_png/united-kingdom.png,//cdn5.f-cdn.com/img/flags/highres_png/united-...,None,...,United Kingdom,None,None,None,None,None,None,None,None,None
4,None,Nashik,in,None,india,/img/flags/png/in.png,//cdn2.f-cdn.com/img/flags/png/in.png,/img/flags/highres_png/india.png,//cdn3.f-cdn.com/img/flags/highres_png/india.png,None,...,India,None,None,None,None,None,None,None,None,None
5,None,London,gb,None,united-kingdom,/img/flags/png/gb.png,//cdn6.f-cdn.com/img/flags/png/gb.png,/img/flags/highres_png/united-kingdom.png,//cdn5.f-cdn.com/img/flags/highres_png/united-...,None,...,United Kingdom,None,None,None,None,None,None,None,None,None
6,None,Torronto,ca,None,canada,/img/flags/png/ca.png,//cdn6.f-cdn.com/img/flags/png/ca.png,/img/flags/highres_png/canada.png,//cdn6.f-cdn.com/img/flags/highres_png/canada.png,None,...,Canada,None,None,None,None,None,None,None,None,None


### primary_currency

In [27]:
%%time
curr = retrieve('about.primary_currency')
print curr.shape

(105880, 6)
CPU times: user 9.94 s, sys: 13 ms, total: 9.96 s
Wall time: 9.93 s


In [28]:
curr.columns=map(lambda x: str(x)+'_2',curr.columns)

In [29]:
curr.head()

,code_2,country_2,exchange_rate_2,id_2,name_2,sign_2
0,USD,US,1,1,US Dollar,$
1,USD,US,1,1,US Dollar,$
4,USD,US,1,1,US Dollar,$
5,USD,US,1,1,US Dollar,$
6,USD,US,1,1,US Dollar,$


### reputation

In [30]:
%%time
repu = retrieve('about.reputation')
print repu.shape

(105880, 65)
CPU times: user 1min 5s, sys: 592 ms, total: 1min 6s
Wall time: 1min 5s


In [31]:
repu.columns=map(lambda x: str(x)+'_4',repu.columns)

In [32]:
repu.head()

,earnings_score_4,entire_history_all_4,entire_history_category_ratings_communication_4,entire_history_category_ratings_expertise_4,entire_history_category_ratings_hire_again_4,entire_history_category_ratings_professionalism_4,entire_history_category_ratings_quality_4,entire_history_complete_4,entire_history_completion_rate_4,entire_history_earnings_4,...,role_4,summary_completion_rate_4,summary_earnings_score_4,summary_earnings_score_pc_4,summary_on_budget_4,summary_on_time_4,summary_rating_score_4,summary_rehire_rate_4,summary_reviews_4,user_id_4
0,0,0,0,0,0,0,0,0,0,None,...,0,N/A,0.0,0,N/A,N/A,0.0,N/A,0,1018449
1,0,0,0,0,0,0,0,0,0,None,...,0,N/A,0.0,0,N/A,N/A,0.0,N/A,0,1344962
4,0,0,0,0,0,0,0,0,0,None,...,0,N/A,0.0,0,N/A,N/A,0.0,N/A,0,1557698
5,0,0,0,0,0,0,0,0,0,None,...,0,N/A,0.0,0,N/A,N/A,0.0,N/A,0,1240449
6,0,0,0,0,0,0,0,0,0,None,...,0,N/A,0.0,0,N/A,N/A,0.0,N/A,0,295694


### status

In [33]:
%%time
stat = retrieve('about.status')
print stat.shape

(105880, 7)
CPU times: user 10.2 s, sys: 16 ms, total: 10.2 s
Wall time: 10.2 s


In [34]:
stat.head()

,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
0,True,True,False,False,False,False,True
1,False,True,False,False,False,False,True
4,False,True,False,False,False,False,True
5,True,True,False,False,True,False,True
6,False,True,False,False,False,False,True


### timezone

In [35]:
%%time
timezone = retrieve('about.timezone')
print timezone.shape

(105880, 4)
CPU times: user 9.01 s, sys: 27 ms, total: 9.04 s
Wall time: 9 s


In [36]:
timezone.columns=map(lambda x: str(x)+'_5',timezone.columns)

In [37]:
timezone.head()

,country_5,id_5,offset_5,timezone_5
0,UK,262,1,Europe/London
1,UK,262,1,Europe/London
4,IN,403,5.5,Asia/Kolkata
5,UK,262,1,Europe/London
6,PK,183,5,Asia/Karachi


### convert list fields to the length of the fields as input since len of the list is not fixed and cannot be unpacked

In [38]:
df2.head(1).iloc[:,-15:]

,about.username_sentence,hourlyRate,resume.certifications,resume.education,resume.exams,resume.experience,resume.publications,resume.show,skills,timeOffset,userId,username,register_Month,register_Year,desc_len
0,Techit99,60.0,"[{u'award_date': u'2011', u'descr': u'Hootsuit...",[],[],"[{u'year_interval': u'-2011', u'month_interval...",[],True,"[{u'usages': 0, u'seo_url': u'php', u'islocal'...",0.0,1018449.0,techit99,March,2009,3


In [39]:
lists=[
'resume.certifications',
'resume.education',
'resume.exams',
'resume.experience',
'resume.publications',
'skills']

In [40]:
def helperLen(x):
    try:
        return len(literal_eval(x))
    except:
        return 0

In [41]:
%%time
for i in lists:
    df2[i+'_len']=df[i].apply(helperLen)

CPU times: user 34.4 s, sys: 58 ms, total: 34.4 s
Wall time: 34.4 s


In [42]:
df3 = df2.drop(lists+['about.location','about.primary_currency','about.reputation','about.status','about.timezone'], axis=1)
df3.shape

(105880, 26)

In [43]:
df3.head()

,about.chosen_role,about.company,about.display_name,about.hourly_rate,about.primary_language,about.public_name,about.recommendations,about.registration_date,about.role,about.tagline,...,username,register_Month,register_Year,desc_len,resume.certifications_len,resume.education_len,resume.exams_len,resume.experience_len,resume.publications_len,skills_len
0,2.0,itsol,techit99,60,en,techit99,0.0,1.237934e+09,1.0,NaN,...,techit99,March,2009,3,1,0,0,1,0,15
1,1.0,Handmade Cards for Animal Charities,jung14,25,en,jung14,0.0,1.262802e+09,0.0,NaN,...,jung14,January,2010,3,0,0,0,0,0,25
4,1.0,Amtech Software,PatilAmit813,10,en,PatilAmit813,0.0,1.273486e+09,0.0,NaN,...,patilamit813,May,2010,3,0,0,0,0,0,9
5,2.0,n/a,tiabil,None,en,tiabil,0.0,1.255351e+09,1.0,NaN,...,tiabil,October,2009,3,0,0,0,0,0,3
6,1.0,John Smith,Smily,3,en,Smily,0.0,1.162855e+09,0.0,NaN,...,smily,November,2006,1934,0,0,0,0,0,25


### concate unpack dictionary fields to the main table

In [47]:
df7 = pd.concat([df3, location, curr, repu, stat, timezone], axis=1)z
df7.shape

(105880, 130)

In [48]:
len(df7['userId'].unique())

105518

In [49]:
len(df7['user_id_4'].unique())

105789

In [50]:
len(df7['username'].unique())

105880

* drop userID and duplicated user ID

In [95]:
df7= df7.drop('user_id_4', axis=1)

In [53]:
df7= df7.drop('userId', axis=1)

In [54]:
df7.shape

(105880, 129)

### check and remove fields have most null values

In [58]:
df7.isnull().sum()[df7.isnull().sum()>50000]

about.tagline                 87250
administrative_area_3        105880
country_demonym_3            105880
country_iso3_3               105880
country_language_code_3      105880
country_language_id_3        105880
country_person_3             105880
country_phone_code_3         105880
country_region_id_3          105880
country_sanction_3           105880
country_seo_url_3            105880
full_address_3               105880
latitude_3                   105880
longitude_3                  105880
vicinity_3                   105880
entire_history_earnings_4    105880
last12months_earnings_4      105880
last3months_earnings_4       105880
project_stats_4              105880
dtype: int64

### drop fields have all null values

In [75]:
all_null_fields = df7.isnull().sum()[df7.isnull().sum()==df7.shape[0]].index
all_null_fields

Index([u'administrative_area_3', u'country_demonym_3', u'country_iso3_3',
       u'country_language_code_3', u'country_language_id_3',
       u'country_person_3', u'country_phone_code_3', u'country_region_id_3',
       u'country_sanction_3', u'country_seo_url_3', u'full_address_3',
       u'latitude_3', u'longitude_3', u'vicinity_3',
       u'entire_history_earnings_4', u'last12months_earnings_4',
       u'last3months_earnings_4', u'project_stats_4'],
      dtype='object')

In [78]:
df7 = df7.drop(all_null_fields, axis=1)
df7.shape

(105880, 111)

### about.tagline should convert to a field that only shows character length

In [64]:
df7['about.tagline'][~df7['about.tagline'].isnull()].head(10)

7        Internet c'est la vie, la vie c'est le travail
15    Professional writer. UK based. Exceptional ski...
18    PMP-VCP-ITIL-CISSP-C|EH-VCP-MCSA-CCNA-RHCE-Linux+
20    Thirteen years of data entry and secretarial e...
23                    "All men of action are dreamers."
25                                  Daty Entry Operator
26                                        Wed developer
36                 Computer Engineer and Web Marketing 
40    We Believe in Accuracy and Customer Satisfaction.
42    Photographer/Videographer/Web Developer/Commun...
Name: about.tagline, dtype: object

In [67]:
df7['about.tagline'] = df7['about.tagline'].apply(lambda x: len(str(x)))

In [82]:
df7 = df7.reset_index(drop=True)

### save file 
### this file parses the json csv, yet did not join with training csv nor do any encoding

In [83]:
df7.to_csv('./goden_full_prepared_7.csv',index=False, encoding='utf-8')